In [ ]:
!ls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Original dataset (41 classes, ~10 images each)
ORIGINAL_DATASET = "/content/drive/MyDrive/cat_datasets/cat_dataset"

# Augmented dataset will be saved here
AUGMENTED_DATASET = "/content/drive/MyDrive/cat_datasets/cat_dataset_augmented"

# Model & checkpoints
MODEL_DIR = "/content/drive/MyDrive/cat_models"


In [ ]:
import os
os.makedirs(AUGMENTED_DATASET, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)


In [ ]:
from torchvision import transforms
from PIL import Image
import os

augment_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
])

TARGET_IMAGES_PER_CLASS = 200


In [ ]:
for class_name in os.listdir(ORIGINAL_DATASET):
    src_class_dir = os.path.join(ORIGINAL_DATASET, class_name)
    dst_class_dir = os.path.join(AUGMENTED_DATASET, class_name)
    os.makedirs(dst_class_dir, exist_ok=True)

    images = os.listdir(src_class_dir)
    count = 0

    while count < TARGET_IMAGES_PER_CLASS:
        for img_name in images:
            img_path = os.path.join(src_class_dir, img_name)
            img = Image.open(img_path).convert("RGB")

            aug_img = augment_transform(img)
            aug_img.save(f"{dst_class_dir}/{class_name}_{count}.png")

            count += 1
            if count >= TARGET_IMAGES_PER_CLASS:
                break

    print(f"Augmented {class_name}: {count} images")


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(AUGMENTED_DATASET, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

NUM_CLASSES = len(dataset.classes)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 112x112
        x = self.pool(F.relu(self.conv2(x)))  # 56x56
        x = self.pool(F.relu(self.conv3(x)))  # 28x28

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [ ]:
model = CustomCNN(NUM_CLASSES).to(device)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

EPOCHS = 20


In [ ]:
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Save checkpoint
    checkpoint_path = f"{MODEL_DIR}/checkpoint_epoch_{epoch+1}.pth"
    torch.save(model.state_dict(), checkpoint_path)

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss:.4f}")
    print(f"Checkpoint saved: {checkpoint_path}")


In [ ]:
correct, total = 0, 0
model.eval()

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


In [ ]:
final_model_path = f"{MODEL_DIR}/custom_cnn_cat_41_classes.pth"
torch.save(model.state_dict(), final_model_path)

print("✅ Final model saved to Drive!")

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/cat_models/custom_cnn_cat_41_classes.pth"
DATASET_PATH = "/content/drive/MyDrive/cat_datasets/cat_dataset_augmented"
IMAGE_PATH = "/content/drive/MyDrive/cat_datasets/cat_dataset/Abyssinian/Abyssinian_10.png"   # change this to your test image


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(DATASET_PATH, transform=transform)
class_names = dataset.classes
num_classes = len(class_names)

print("Classes loaded:", num_classes)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CustomCNN(num_classes).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

print("✅ Model loaded successfully")


In [ ]:
from PIL import Image

img = Image.open(IMAGE_PATH).convert("RGB")
img_tensor = transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    output = model(img_tensor)
    probs = torch.softmax(output, dim=1)
    confidence, pred = torch.max(probs, dim=1)

print("Predicted Class:", class_names[pred.item()])
print(f"Confidence: {confidence.item()*100:.2f}%")

img
